# Functions for USGS Louisiana Gage Data Extraction 

This Jupyter notebook contains a collection of Python functions designed to support the analysis and visualization of USGS gage data. These functions are intended to be used in conjunction with the `USGS_Louisiana_Gage_Data_Extraction.ipynb` to keep the results notebook clean and user-friendly.

## Overview
- **Data Handling Functions**: Functions for reading, reprojecting, and clipping geospatial data.
- **Data Retrieval Functions**: Functions to fetch stage and discharge data from the USGS NWIS service.
- **Visualization Functions**: Functions to generate plots for visualizing stage and discharge data over time.

## Usage
1. **Function Definition**: Each cell in this notebook defines a specific function along with its documentation.
2. **Execution**: Execute all cells to make the functions available for use in the `USGS_Louisiana_Gage_Data_Extraction.ipynb`.
3. **Integration**: Use the `%run` magic command in the results notebook to import and utilize these functions.

Note: This notebook should be executed before running the `USGS_Louisiana_Gage_Data_Extraction.ipynb` to ensure all functions are available for use.


In [ ]:
import requests
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
def fetch_usgs_data_for_louisiana():
    """
    Fetches data for active USGS sites in Louisiana from the USGS NWIS service.

    This function retrieves site data in 'rdb' format, focusing on sites within the state of Louisiana that are currently active.
    The returned data includes the site number, site name, latitude, and longitude for each site.

    Returns:
        list of tuple: A list of tuples, where each tuple contains four elements:
            - site_number (str): The site's unique identifier.
            - site_name (str): The name of the site.
            - latitude (str): The latitude of the site.
            - longitude (str): The longitude of the site.

    Note:
        If there's an issue fetching the data, the function prints an error message and returns None.
    """
    BASE_URL = "https://waterservices.usgs.gov/nwis/site/"
    PARAMS = {
        "format": "rdb",
        "stateCd": "LA",
        "siteStatus": "active"
    }

    response = requests.get(BASE_URL, params=PARAMS)

    if response.status_code != 200:
        print("Error fetching data from USGS.")
        return

    lines = response.text.split("\n")
    sites = []

    for line in lines:
        if not line.startswith("#") and line:  # ignore comments and empty lines
            data = line.split("\t")
            if len(data) > 5:  # ensure there's enough data to unpack
                site_number, site_name, latitude, longitude = data[1], data[2], data[4], data[5]
                sites.append((site_number, site_name, latitude, longitude))

    return sites

In [ ]:
def plot_usgs_gages_on_map(clipped_gages, df_usgs_discharge):
    '''Creates an interactive map displaying USGS gages with their average discharge values, omitting gages with NaN mean_discharge values.
    
    Args:
        clipped_gages (GeoDataFrame): The geospatial data of the gages.
        df_usgs_discharge (DataFrame): The discharge data for the gages.
        
    Returns:
        folium.Map: The interactive map object.
    '''
    
    import folium
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    
    # Calculate the center of your data for initializing the map view:
    latitude_center = clipped_gages.geometry.y.mean()
    longitude_center = clipped_gages.geometry.x.mean()

    # Create a map object:
    m = folium.Map(location=[latitude_center, longitude_center], zoom_start=7)

    # Add the average discharge data to the clipped_gages dataframe
    def get_mean_discharge(site):
        key = f"{site}, USGS"
        return df_usgs_discharge[key].mean() if key in df_usgs_discharge.columns else np.nan

    clipped_gages['mean_discharge'] = clipped_gages['Site Number'].apply(get_mean_discharge)

    # Drop rows with NaN mean_discharge values
    clipped_gages = clipped_gages.dropna(subset=['mean_discharge'])

    # Set the colormap normalization based on the range of mean_discharge values
    norm = mcolors.Normalize(vmin=clipped_gages['mean_discharge'].min(), vmax=clipped_gages['mean_discharge'].max())

    # Plot each point from the DataFrame to the map:
    for _, row in clipped_gages.iterrows():
        # Assign color based on mean_discharge value:
        color = plt.cm.viridis(norm(row['mean_discharge']))
        color = mcolors.to_hex(color)

        folium.CircleMarker(
            location=[row['geometry'].y, row['geometry'].x],
            radius=5,
            popup=f"Site: {row['Site Number']}, Avg Discharge: {row['mean_discharge']:.2f} ft³/s",
            color=color,
            fill=True,
            fill_color=color
        ).add_to(m)

    return m

In [ ]:

def process_usgs_site_data(sites):
    """
    Processes the fetched USGS site data for Louisiana, extracts longitudes and latitudes, and converts them to a GeoDataFrame.

    The script carries out the following steps:
    1. Fetch USGS site data for Louisiana.
    2. Extract longitude and latitude values from the site data.
    3. Convert longitude and latitude string values to numeric format. Invalid parsing results in NaN values.
    4. Filters out sites without valid numerical latitude and longitude values.
    5. Converts the valid sites' data into a GeoDataFrame with corresponding geometries.
    6. Sets the initial Coordinate Reference System (CRS) to WGS84 (EPSG:4326) and then reprojects the data to NAD27 (EPSG:4267).

    Attributes:
        sites (list of tuple): Fetched USGS site data.
        longitudes (list of str): Extracted longitudes from the site data.
        latitudes (list of str): Extracted latitudes from the site data.
        longitudes_numeric (Series): Longitudes converted to numeric format.
        latitudes_numeric (Series): Latitudes converted to numeric format.
        valid_sites (list of tuple): Filtered sites having valid numerical latitude and longitude values.
        geometry (list of Point): List of Point geometries corresponding to each valid site.
        gdf (GeoDataFrame): GeoDataFrame constructed from valid sites.
        gdf_nad27 (GeoDataFrame): GeoDataFrame reprojected to NAD27 (EPSG:4267) CRS.
    """
    sites = fetch_usgs_data_for_louisiana()

    # Extract the longitude and latitude values
    longitudes = [site[3] for site in sites]
    latitudes = [site[2] for site in sites]

    # Convert string values to numeric, setting errors='coerce' to turn invalid parsing into NaN
    longitudes_numeric = pd.to_numeric(longitudes, errors='coerce')
    latitudes_numeric = pd.to_numeric(latitudes, errors='coerce')

    # Filter out any sites that don't have valid numerical latitude and longitude
    valid_sites = [sites[i] for i in range(len(sites)) if not (pd.isna(longitudes_numeric[i]) or pd.isna(latitudes_numeric[i]))]

    # Convert valid sites data to GeoDataFrame
    geometry = [Point(xy) for xy in zip(pd.to_numeric([site[3] for site in valid_sites]), pd.to_numeric([site[2] for site in valid_sites]))]
    gdf = gpd.GeoDataFrame(valid_sites, columns=['Site Number', 'Site Name', 'Latitude', 'Longitude'], geometry=geometry)

    # Set the initial CRS to WGS84 (EPSG:4326) and then reproject to NAD27 (EPSG:4267)
    gdf.crs = "EPSG:4326"
    gdf_nad27 = gdf.to_crs("EPSG:4267")


In [ ]:

def visualize_usgs_site_data_on_map(gdf_nad27):
    
    """
    Visualizes the USGS site data (reprojected to NAD27) on a map with a basemap background.

    The script carries out the following steps:
    1. Plots the `gdf_nad27` GeoDataFrame, which contains the reprojected USGS site data.
    2. Adds a basemap in the background using the `contextily` library.
    3. Displays the map with the USGS site data points.

    Attributes:
        gdf_nad27 (GeoDataFrame): GeoDataFrame containing the USGS site data reprojected to NAD27 (EPSG:4267) CRS.

    Note:
        Ensure that the `geopandas`, `matplotlib`, and `contextily` libraries are imported, and that `gdf_nad27` is already defined and populated before executing the script.
    """


    # Plot the USGS site data
    ax = gdf_nad27.to_crs(epsg=3857).plot(figsize=(10, 10), markersize=5, color='red', alpha=0.7)

    # Add a basemap in the background
    ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

    # Set map title and axis labels
    ax.set_title('USGS Site Data in Louisiana')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_axis_off()

    plt.tight_layout()
    plt.show()

    # Implementation of the function (Actual code for visualizing USGS site data on map)


In [ ]:
def clip_gages_to_huc8_boundary(huc8_path: str, gdf_nad27: gpd.GeoDataFrame) -> (gpd.GeoDataFrame, gpd.GeoDataFrame):
    """
    Loads a HUC8 boundary shapefile, reprojects it to match the CRS of the USGS gages, and then clips the gages to the HUC8 boundary.
    
    Parameters:
        huc8_path (str): The path to the HUC8 boundary shapefile.
        gdf_nad27 (gpd.GeoDataFrame): GeoDataFrame containing the USGS gages data.
        
    Returns:
        huc8_gdf_reprojected (gpd.GeoDataFrame): The HUC8 boundary data reprojected to NAD27 (EPSG:4267) CRS.
        clipped_gages (gpd.GeoDataFrame): The USGS gages data clipped to the HUC8 boundary.
    """
    # Read the HUC8s shapefile using geopandas
    huc8_gdf = gpd.read_file(huc8_path)
    
    # Reproject the HUC8s shapefile to match the CRS of the gages (NAD27)
    huc8_gdf_reprojected = huc8_gdf.to_crs(gdf_nad27.crs)
    
    # Use geopandas' clip function to clip the gages to the reprojected HUC8s boundary
    clipped_gages = gpd.clip(gdf_nad27, huc8_gdf_reprojected)
    
    return huc8_gdf_reprojected, clipped_gages


In [ ]:

def visualize_clipped_usgs_gages_with_huc8_boundary(huc8_gdf_reprojected, clipped_gages):
    """
    Visualizes the clipped USGS gages data on a static map, overlaying the HUC8 boundary.

    The script carries out the following steps:
    1. Initializes a plotting figure with specified dimensions.
    2. Plots the reprojected HUC8 boundary on the figure as a background layer.
    3. Overlays the clipped USGS gages data on top of the HUC8 boundary.
    4. Sets the map title, x-axis and y-axis labels, and a legend indicating the respective data layers.
    5. Displays the resulting map.

    Attributes:
        huc8_gdf_reprojected (GeoDataFrame): GeoDataFrame containing the HUC8 boundary data reprojected to NAD27 (EPSG:4267) CRS.
        clipped_gages (GeoDataFrame): GeoDataFrame containing the USGS gages data clipped to the HUC8 boundary.

    Note:
        Ensure that the `matplotlib` and `geopandas` libraries are imported, and that both `huc8_gdf_reprojected` and `clipped_gages` are defined and populated before executing the script.
    """
    # Initialize a figure and axis for the plot
    fig, ax = plt.subplots(figsize=(12, 10))

    # Plot the HUC8 boundary as the background
    huc8_gdf_reprojected.plot(ax=ax, color='lightgray', edgecolor='black', label='HUC8 Boundary')

    # Plot the clipped USGS site data on top
    clipped_gages.plot(ax=ax, markersize=50, color='red', alpha=0.7, label='USGS Gages')

    # Set map title, legend, and axis labels
    ax.set_title('Clipped USGS Site Data with HUC8 Boundary')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

   

In [ ]:
def fetch_and_process_usgs_data(START_DATE: str, END_DATE: str, clipped_gages: gpd.GeoDataFrame, datum_corr_list: list = None) -> (pd.DataFrame, pd.DataFrame):
    """
    Fetches stage and discharge data for a list of USGS gages within a specified date range and processes the retrieved data.
    
    Parameters:
        START_DATE (str): The start date for data retrieval in the format 'YYYY-MM-DD'.
        END_DATE (str): The end date for data retrieval in the format 'YYYY-MM-DD'.
        clipped_gages (gpd.GeoDataFrame): GeoDataFrame containing the clipped USGS gages data.
        datum_corr_list (list of float, optional): List of datum corrections for each site. Initialized with zeros if not provided.
        
    Returns:
        df_usgs_stage (pd.DataFrame): Consolidated DataFrame containing stage data for all sites.
        df_usgs_discharge (pd.DataFrame): Consolidated DataFrame containing discharge data for all sites.
    """
    # Extract site numbers from clipped_gages
    site_list = clipped_gages["Site Number"].tolist()

    # Initialize datum_corr_list with zeros if not provided
    if datum_corr_list is None:
        datum_corr_list = [0] * len(site_list)

    # Initialize lists to store DataFrames for stage and discharge data
    stage_dfs = []
    discharge_dfs = []

    # Fetch the data for each site
    for site in site_list:
        site_name = f"{site}, USGS"
        try:
            # Use nwis to fetch the data
            data = nwis.get_record(sites=site, service='iv', start=START_DATE, end=END_DATE)
            data.index = pd.to_datetime(data.index, utc=True).tz_convert(tz="America/Chicago")
            data = data.groupby(by=pd.Grouper(freq='D')).mean(numeric_only=True)
            
            # Check if datum correction is available for the site, else default to 0
            datum_corr = datum_corr_list[site_list.index(site)]
            
            # Extract stage and discharge data and store in the list
            if '00065' in data.columns:
                stage_dfs.append(data[['00065']].rename(columns={'00065': site_name}) + datum_corr)
            if '00060' in data.columns:
                discharge_dfs.append(data[['00060']].rename(columns={'00060': site_name}))
                
        except Exception as e:
            print(f"Error encountered for site {site_name}: {e}")

    # Concatenate all DataFrames in the list
    df_usgs_stage = pd.concat(stage_dfs, axis=1)
    df_usgs_discharge = pd.concat(discharge_dfs, axis=1)

    return df_usgs_stage, df_usgs_discharge


In [ ]:
def examine_usgs_data(df_usgs_stage: pd.DataFrame, df_usgs_discharge: pd.DataFrame) -> None:
    """
    Examines and displays a summary of the fetched stage and discharge data for USGS gages.
    
    Parameters:
        df_usgs_stage (pd.DataFrame): DataFrame containing consolidated stage data for USGS gages.
        df_usgs_discharge (pd.DataFrame): DataFrame containing consolidated discharge data for USGS gages.
    """
    # Examine stage data
    if not df_usgs_stage.empty:
        print("Stage Data:")
        print(df_usgs_stage.head())
        print("\nStage Data Info:")
        print(df_usgs_stage.info())
        print("\nStage Data Statistics:")
        print(df_usgs_stage.describe())
    else:
        print("No stage data available.")
    
    # Examine discharge data
    if not df_usgs_discharge.empty:
        print("\nDischarge Data:")
        print(df_usgs_discharge.head())
        print("\nDischarge Data Info:")
        print(df_usgs_discharge.info())
        print("\nDischarge Data Statistics:")
        print(df_usgs_discharge.describe())
    else:
        print("\nNo discharge data available.")

In [ ]:
import matplotlib.pyplot as plt

def plot_stage_data_for_site(df_usgs_stage: pd.DataFrame, site_number: str) -> None:
    """
    Plots the stage data over time for a specific USGS gage site.
    
    Parameters:
        df_usgs_stage (pd.DataFrame): DataFrame containing consolidated stage data for USGS gages.
        site_number (str): The site number of the USGS gage site to be plotted.
    """
    # Check if the specified site number exists in the DataFrame
    site_column = f"{site_number}, USGS"
    if site_column not in df_usgs_stage.columns:
        print(f"No data available for {site_column}")
        return
    
    # Set the figure size for the plot
    plt.figure(figsize=(14, 6))
    
    # Plot the stage data for the site against the dates
    plt.plot(df_usgs_stage.index, df_usgs_stage[site_column], label=site_column)
    
    # Set the title, x-axis label, y-axis label, and legend for the plot
    plt.title(f'Stage Data Over Time for {site_column}')
    plt.xlabel('Date')
    plt.ylabel('Stage (ft)')
    plt.legend(loc='upper left')
    
    # Add grid lines for better readability
    plt.grid(True)
    
    # Adjust the layout for optimal display
    plt.tight_layout()
    
    # Display the plot
    plt.show()

In [ ]:
def plot_discharge_data_for_site(df_usgs_discharge: pd.DataFrame, site_number: str) -> None:
    """
    Plots the discharge data over time for a specific USGS gage site.
    
    Parameters:
        df_usgs_discharge (pd.DataFrame): DataFrame containing consolidated discharge data for USGS gages.
        site_number (str): The site number of the USGS gage site to be plotted.
    """
    # Check if the specified site number exists in the DataFrame
    site_column = f"{site_number}, USGS"
    if site_column not in df_usgs_discharge.columns:
        print(f"No data available for {site_column}")
        return
    
    # Set the figure size for the plot
    plt.figure(figsize=(14, 6))
    
    # Plot the discharge data for the site against the dates
    plt.plot(df_usgs_discharge.index, df_usgs_discharge[site_column], label=site_column)
    
    # Set the title, x-axis label, y-axis label, and legend for the plot
    plt.title(f'Discharge Data Over Time for {site_column}')
    plt.xlabel('Date')
    plt.ylabel('Discharge ($ft^3/s$)')
    plt.legend(loc='upper left')
    
    # Add grid lines for better readability
    plt.grid(True)
    
    # Adjust the layout for optimal display
    plt.tight_layout()
    
    # Display the plot
    plt.show()


In [ ]:
def plot_all_stage_data(df_usgs_stage: pd.DataFrame) -> None:
    """
    Plots the stage data over time for all USGS gage sites present in the `df_usgs_stage` DataFrame.
    
    Parameters:
        df_usgs_stage (pd.DataFrame): DataFrame containing consolidated stage data for USGS gages.
    """
    # Check if the DataFrame is empty
    if df_usgs_stage.empty:
        print("No stage data available.")
        return
    
    # Set the figure size for the plot
    plt.figure(figsize=(20, 10))
    
    # Iterate over each column in the DataFrame, plotting the stage data for each USGS gage site
    for column in df_usgs_stage.columns:
        plt.plot(df_usgs_stage.index, df_usgs_stage[column], label=column)
    
    # Set the title, x-axis label, y-axis label, and legend for the plot
    plt.title('Stage Data Over Time for All Gages')
    plt.xlabel('Date')
    plt.ylabel('Stage (ft)')
    
    # Adjust the y-axis limits to accommodate the maximum stage value across all gages with some buffer
    plt.ylim(0, df_usgs_stage.max().max() + 10)
    
    # Add grid lines for better readability
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.grid(True)
    
    # Display the plot
    plt.show()

In [ ]:
def plot_all_discharge_data(df_usgs_discharge: pd.DataFrame) -> None:
    """
    Plots the discharge data over time for all USGS gage sites present in the `df_usgs_discharge` DataFrame.
    
    Parameters:
        df_usgs_discharge (pd.DataFrame): DataFrame containing consolidated discharge data for USGS gages.
    """
    # Check if the DataFrame is empty
    if df_usgs_discharge.empty:
        print("No discharge data available.")
        return
    
    # Set the figure size for the plot
    plt.figure(figsize=(20, 10))
    
    # Iterate over each column in the DataFrame, plotting the discharge data for each USGS gage site
    for column in df_usgs_discharge.columns:
        plt.plot(df_usgs_discharge.index, df_usgs_discharge[column], label=column)
    
    # Set the title, x-axis label, y-axis label, and legend for the plot
    plt.title('Discharge Data Over Time for All Gages')
    plt.xlabel('Date')
    plt.ylabel('Discharge ($ft^3/s$)')
    
    # Add grid lines for better readability
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.grid(True)
    
    # Adjust the layout for optimal display
    plt.tight_layout()
    
    # Display the plot
    plt.show()